In [ ]:
#Notebook to accompany Microsoft AI Search's "Advanced RAG with Azure AI Search and LlamaIndex" Blog

In [ ]:
###Environment Prerequisites prior to running this notebook

##Provision Azure Open AI Service:https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource
    #Create an embeddings deployment (text-embeddings-ada002): https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#embeddings-models
    #Create an LLM model deployment (gpt-4): https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#gpt-4-and-gpt-4-turbo-preview
    #Once the deployments are created, take note of the endpoint and key for the service and the deployment names for each model

##Provision an Azure AI Search Instance: https://learn.microsoft.com/en-us/azure/search/search-create-service-portal
    #Choose the tier with semantic ranker enabled

##Install Python 3.8 or later: https://www.python.org/downloads/

##Install Jupyter Notebook: https://jupyter.org/install

In [ ]:
####Install required packages
%pip install ipykernel azure-search-documents llama-index llama-index-vector-stores-azureaisearch llama-index-embeddings-azure-openai azure-identity

In [2]:
#import modules
import os
from IPython.display import Markdown, display
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement, MetadataIndexFieldType
from llama_index.core import StorageContext, VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential

In [20]:
#***Fill-in Azure Open AI Variables****
#Obtain Azure OpenAI (AOAI) key & endpoint in the Azure Portal -> Azure OpenAI -> AOAI Instance -> Resource Management -> Keys and Endpoint
AOAI_API_KEY = 'your alphanumeric AOAI key'
AOAI_ENDPOINT = 'https://youraoai.openai.azure.com/'

#AOAI api version below should work for gpt-4 models
AOAI_API_VERSION = '2024-02-15-preview'

#Change if you want to use a different model, ensure you use the 'model name' and not the 'deployment name' from the Azure OpenAI Service
AOAI_LLM_MODEL = 'gpt-4'

#Change the deployment name of your AOAI llm model if not the same as the model name
AOAI_LLM_DEPLOYMENT_NAME = "gpt-4"

#The Azure OpenAI Embedding model is a separate model that is used to convert text to embeddings, usually text-embedding-ada-002
AOAI_EMBEDDING_MODEL = "text-embedding-ada-002"

#Change to the 'deployment name' of your embeddings model if not same as the model name
AOAI_EMBEDDING_DEPLOYMENT_NAME = "text-embedding-ada-002"

#***Create Azure AI Search Variables
#Obtain from Azure Portal -> AI Search -> Settings -> Keys
SEARCH_SERVICE_API_KEY = "your AI Search key"
#Obtain from Azure Portal -> AI Search -> Overview -> Url 
SEARCH_SERVICE_ENDPOINT = "https://youraisearchurl.search.windows.net"
#If needed to change, versions table here: https://learn.microsoft.com/en-us/rest/api/searchservice/search-service-api-versions#stable-versions
SEARCH_SERVICE_API_VERSION = "2023-11-01"
CREDENTIAL = AzureKeyCredential(SEARCH_SERVICE_API_KEY)
#Name the index to be created in AI Search (lowercase, digits or dashes only)
INDEX_NAME = "llamaindex-vector-demo"
###*****End of Variables*****####

llm = AzureOpenAI(
    model=AOAI_LLM_MODEL,
    deployment_name=AOAI_LLM_DEPLOYMENT_NAME,
    api_key=AOAI_API_KEY,
    azure_endpoint=AOAI_ENDPOINT,
    api_version=AOAI_API_VERSION,
)

embed_model = AzureOpenAIEmbedding(
    model=AOAI_EMBEDDING_MODEL,
    deployment_name=AOAI_EMBEDDING_DEPLOYMENT_NAME,
    api_key=AOAI_API_KEY,
    azure_endpoint=AOAI_ENDPOINT,
    api_version=AOAI_API_VERSION,
)


In [21]:
###AI Search Configuration

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=SEARCH_SERVICE_ENDPOINT,
    credential=CREDENTIAL,
)

# Use search client to demonstration using existing index
search_client = SearchClient(
    endpoint=SEARCH_SERVICE_ENDPOINT,
    index_name=INDEX_NAME,
    credential=CREDENTIAL,
)

metadata_fields = {
    "author": "author",
    "theme": ("topic", MetadataIndexFieldType.STRING),
    "director": "director",
}


In [22]:
###Vector store configuration
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    filterable_metadata_field_keys=metadata_fields,
    index_name=INDEX_NAME,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1536,
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
)


In [23]:
###Automatically download and index the sample dataset to the working directory
import requests
url = 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt'
response = requests.get(url)
with open('paul_graham_essay.txt', 'wb') as f:
    f.write(response.content)

reader = SimpleDirectoryReader(input_files=["paul_graham_essay.txt"])
documents = reader.load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Settings.llm = llm
Settings.embed_model = embed_model
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)


In [25]:
# Query Data - First query can take up to 5 minutes depending on the size of the dataset to index
query_engine = index.as_query_engine(similarity_top_k=3)
response = query_engine.query("What did the author work on?")
display(Markdown(f"{response}"))

The author worked on several different things throughout his life. Before college, he worked on writing short stories and programming. His first programming experiences were with an IBM 1401 using punch cards and Fortran. Later, he worked with microcomputers, writing simple games, a program to predict model rocket flights, and a word processor his father used for writing a book.

In college, he initially planned to study philosophy but switched to AI, inspired by a novel by Heinlein and a PBS documentary featuring SHRDLU. He also worked on Lisp, creating a new dialect called Arc, and wrote essays that he published online, which gained significant attention after being shared on Slashdot. He wrote a book about Lisp hacking titled "On Lisp," and engaged in systems work, desiring to build things that would last.

Additionally, he worked on spam filters, did some painting, hosted dinners, and bought a building in Cambridge. He also contemplated a career in art after realizing paintings could be a lasting form of creation, taking art classes while still in a PhD program for computer science. Ultimately, he completed his dissertation in a short time frame after being asked if he could graduate that June.